# Analyse des températures maximales journalières : une plongée dans les données météorologiques françaises


L'analyse présentée dans ce notebook s'appuie sur un jeu de données météorologiques quotidiennes pour diverses stations en France, disponible à l'adresse suivante : https://object.files.data.gouv.fr/meteofrance/data/synchro_ftp/BASE/QUOT/Q_95_latest-2024-2025_RR-T-Vent.csv.gz. Ce fichier contient des informations précieuses sur les précipitations, les températures, les vents, et d'autres paramètres météorologiques. L'objectif principal de cette analyse est de préparer les données pour examiner les températures maximales journalières, calculer la moyenne glissante sur 30 jours, et identifier les anomalies par comparaison avec cette moyenne glissante, puis de lisser ces anomalies sur 7 jours.


L'étude des températures maximales journalières et de leurs anomalies est cruciale pour comprendre les modèles climatiques et météorologiques en France. En analysant ces données, nous pouvons mettre en lumière les tendances et les variations significatives dans les températures, contribuant ainsi à une meilleure compréhension du climat et à une prise de décision éclairée dans divers domaines tels que l'agriculture, l'urbanisme et la gestion des ressources.


## Méthodologie

La méthodologie employée pour cette analyse implique plusieurs étapes clés. Tout d'abord, les données sont chargées et préparées à l'aide de requêtes SQL dans DuckDB, permettant un traitement efficace et rapide des grandes quantités de données. Les températures moyennes journalières sont calculées et utilisées pour déterminer la moyenne glissante sur 30 jours. Ensuite, les anomalies sont identifiées en comparant les températures journalières avec cette moyenne glissante. Enfin, ces anomalies sont lissées sur 7 jours pour atténuer les fluctuations quotidiennes et mettre en évidence les tendances sous-jacentes. Les résultats sont ensuite visualisés à l'aide de Plotly pour une représentation graphique claire et interactive des données et des analyses effectuées.

## 🔧 Configuration

In [1]:
# Installation et imports
import duckdb as ddb
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

## 🦆 Chargement du dataset avec Duckdb

In [1]:
# Fonction de chargement complète (basée sur load_file_from_url_lite)
def load_file_from_url_lite(url_dataset="", loader="read_csv_auto", options="", nom_table="loaded_dataset", safe_mode=False):
    ddb.execute("install spatial")
    ddb.execute("load spatial")
    ddb.execute("INSTALL h3 FROM community")
    ddb.execute("LOAD h3")
    ddb.execute("install webbed from community;")
    ddb.execute("load webbed")
    ddb.execute("set force_download=True")
    ddb.execute(f"drop table if exists {nom_table}")   
    
    # Détection automatique du type de fichier
    if 'csv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'tsv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'txt' in url_dataset: 
        loader = "read_csv_auto"
    elif 'parquet' in url_dataset: 
        loader = "read_parquet"
    elif 'json' in url_dataset: 
        loader = "read_json_auto"
    elif 'xls' in url_dataset or 'xlsx' in url_dataset: 
        loader = "st_read"
    elif 'shp' in url_dataset: 
        loader = "st_read"
    elif 'geojson' in url_dataset: 
        loader = "st_read"
    elif 'xml' in url_dataset: 
        loader = "read_xml"
    elif 'html' in url_dataset: 
        loader = "read_html"
    else: 
        raise ValueError(f"Type de fichier non supporté pour {url_dataset}")
    
    if options=="": 
        options = "" 
    if 'csv' in url_dataset and safe_mode==True: 
        options = ", all_varchar=1" 
    if nom_table=="": 
        nom_table = "loaded_dataset"
    
    try:
        status = ddb.sql(f"""
            create or replace table {nom_table} as select *
            from
            {loader}("{url_dataset}" {options})
        """)
        return status
    except Exception as e:
        return f"Erreur au chargement du fichier : {str(e)}"

def run_query(sql):
    return ddb.sql(sql.replace("`"," ")).to_df()

# Chargement des données
load_file_from_url_lite("https://object.files.data.gouv.fr/meteofrance/data/synchro_ftp/BASE/QUOT/Q_95_latest-2024-2025_RR-T-Vent.csv.gz", safe_mode=True)
print("✅ Données chargées avec succès")

## 🔍 Analyse SQL

Cette requête utilise des techniques SQL pour extraire et transformer les données de manière efficace.

In [2]:
# Exécution de la requête
df = run_query(""" WITH 
daily_avg AS (
  SELECT 
    CAST(SUBSTRING("AAAAMMJJ", 1, 4) AS INTEGER) AS year,
    CAST(SUBSTRING("AAAAMMJJ", 5, 2) AS INTEGER) AS month,
    CAST(SUBSTRING("AAAAMMJJ", 7, 2) AS INTEGER) AS day,
    CAST("TM" AS DOUBLE) AS "TM",
    "AAAAMMJJ"
  FROM 
    loaded_dataset
),
avg_tm AS (
  SELECT 
    "AAAAMMJJ",
    AVG("TM") AS avg_tm_day
  FROM 
    daily_avg
  GROUP BY 
    "AAAAMMJJ"
),
moving_avg AS (
  SELECT 
    "AAAAMMJJ",
    avg_tm_day,
    AVG(avg_tm_day) OVER (ORDER BY "AAAAMMJJ" ROWS BETWEEN 29 PRECEDING AND CURRENT ROW) AS moving_avg_30_days
  FROM 
    avg_tm
),
anomalies AS (
  SELECT 
    "AAAAMMJJ",
    avg_tm_day,
    moving_avg_30_days,
    avg_tm_day - moving_avg_30_days AS anomaly
  FROM 
    moving_avg
)
SELECT 
  "AAAAMMJJ",
  avg_tm_day,
  moving_avg_30_days,
  anomaly,
  AVG(anomaly) OVER (ORDER BY "AAAAMMJJ" ROWS BETWEEN 3 PRECEDING AND 3 FOLLOWING) AS smoothed_anomaly_7_days
FROM 
  anomalies
ORDER BY 
  "AAAAMMJJ" """)
print(f"Résultats : {len(df)} lignes")
df.head()

## 📈 Visualisation

La bibliothèque principale utilisée est Plotly, qui est idéale pour créer des visualisations interactives et personnalisables. Plotly est adaptée pour représenter des données temporelles complexes avec plusieurs courbes et des info-bulles détaillées. Elle permet une représentation claire et dynamique des données de température journalière et de leurs anomalies.

In [3]:
import pandas as pd
import duckdb as ddb
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

df['AAAAMMJJ'] = pd.to_datetime(df['AAAAMMJJ'], format='%Y%m%d')

dataviz = go.Figure()

dataviz.add_trace(go.Scatter(x=df['AAAAMMJJ'], y=df['avg_tm_day'],
                             mode='lines',
                             name='Température journalière',
                             line=dict(color='lightgrey'), hovertemplate='%{y:.2f}°C<br>%{x|%Y-%m-%d}<extra></extra>'))

dataviz.add_trace(go.Scatter(x=df['AAAAMMJJ'], y=df['moving_avg_30_days'],
                             mode='lines',
                             name='Moyenne glissante 30 jours',
                             line=dict(color='red'), hovertemplate='%{y:.2f}°C<br>%{x|%Y-%m-%d}<extra></extra>'))

dataviz.add_trace(go.Scatter(x=df['AAAAMMJJ'], y=df['smoothed_anomaly_7_days'],
                             mode='lines',
                             name='Anomalie lissée 7 jours',
                             line=dict(color='teal', dash='dot'), hovertemplate='%{y:.2f}°C<br>%{x|%Y-%m-%d}<extra></extra>'))

dataviz.update_layout(
    height=500,
    margin=dict(l=20, r=20, t=60, b=20),
    hovermode='x unified',
    font=dict(family='Source Sans Pro', size=12),
    title=dict(text="Analyse des températures maximales journalières et anomalies", x=0.5),
    xaxis_title="Date",
    yaxis_title="Température (°C)",
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)
dataviz

---
*Made with ❤️ and with [duckit.fr](https://duckit.fr) - [Ali Hmaou](https://www.linkedin.com/in/ali-hmaou-6b7b73146/)*